## Notebook for working with shapely files

### Opened Tuesday 12-13

#### Couldn't create geopanda df correctly when I imported from a csv so the first few cells are all of the transformations from the clean_parse notebook that created a working object

In [13]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

file_path = ('data\kml\My Places_dec_9_22.kml')

with open(file_path) as file:

    xml_data = file.read()

# Initializing soup variables
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'fop'])



## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Clean up the data

# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]


## remove numberic characters and . from field names
df['field'] = df['field'].str.replace(r'\d+', '')
    

C:\Users\Justin\AppData\Local\Temp\ipykernel_15628\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_15628\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_15628\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_15628\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Loca

In [14]:

## Create a (lat, long) pair for home plate 
## Just grab first uple form the foul ground coordinates

df['home_plate'] = df['foul'].str.split(' ').str[0]

## Calculate the area of each polygon in square meters
df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))
df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

# Convert the area to square feet
df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)

# drop the area columns
df = df.drop(['foul_area', 'fop_area'], axis=1)

# calculate the ratio of foul ground to total area
df['foul_pct'] = (df['foul_ft']/(df['fop_ft']+df['foul_ft']))*100

## Convert the coordinates to a list of tuples
df['foul'] = df['foul'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])
df['fop'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])


## Create shapely polygons from the coordinates

from shapely.geometry import Polygon
import geopandas as gpd

df['foul'] = df['foul'].apply(lambda x: Polygon(x))
df['fop'] = df['fop'].apply(lambda x: Polygon(x))


df.head()


,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct
0,. Beal City,"POLYGON Z ((-84.9127615 43.6741539 0, -84.9127...","POLYGON Z ((-84.9127615 43.6741539 0, -84.9115...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183
1,. Buchanan,"POLYGON Z ((-86.3709785 41.8239409 0, -86.3717...","POLYGON Z ((-86.3709785 41.8239409 0, -86.3700...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361
2,. Grosse Pointe Woods University Liggett,"POLYGON Z ((-82.9007499 42.4260393 0, -82.9002...","POLYGON Z ((-82.9007499 42.4260393 0, -82.8997...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835
3,. Jackson Lumen Christi,"POLYGON Z ((-84.4602665 42.2219816 0, -84.4603...","POLYGON Z ((-84.4602665 42.2219816 0, -84.4591...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358
4,. Kalamazoo Christian,"POLYGON Z ((-85.6134503 42.273406 0, -85.61346...","POLYGON Z ((-85.6134503 42.273406 0, -85.61452...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027


## end of code from clean_notebook_parse

### returning usualble gdf

In [72]:
df.info()
from shapely.geometry import Point

## Convert the home plate coordinate into a shapely point
df['home_pt'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))
df['home_pt'] = df['home_pt'].apply(lambda x: x.buffer(0.0001))
 



df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 0 to 246
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   field       222 non-null    object 
 1   foul        222 non-null    object 
 2   fop         222 non-null    object 
 3   home_plate  222 non-null    object 
 4   foul_ft     222 non-null    float64
 5   fop_ft      222 non-null    float64
 6   foul_pct    222 non-null    float64
 7   home_lat    222 non-null    object 
 8   home_long   222 non-null    object 
 9   home_pt     222 non-null    object 
dtypes: float64(3), object(7)
memory usage: 27.2+ KB


,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct,home_lat,home_long,home_pt
0,. Beal City,"POLYGON Z ((-84.9127615 43.6741539 0, -84.9127...","POLYGON Z ((-84.9127615 43.6741539 0, -84.9115...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183,-84.9127615,43.6741539,"POLYGON ((-84.9126615 43.6741539, -84.91266198..."
1,. Buchanan,"POLYGON Z ((-86.3709785 41.8239409 0, -86.3717...","POLYGON Z ((-86.3709785 41.8239409 0, -86.3700...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361,-86.37097850000001,41.8239409,"POLYGON ((-86.3708785 41.8239409, -86.37087898..."
2,. Grosse Pointe Woods University Liggett,"POLYGON Z ((-82.9007499 42.4260393 0, -82.9002...","POLYGON Z ((-82.9007499 42.4260393 0, -82.8997...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835,-82.90074989999999,42.4260393,"POLYGON ((-82.90064989999999 42.4260393, -82.9..."
3,. Jackson Lumen Christi,"POLYGON Z ((-84.4602665 42.2219816 0, -84.4603...","POLYGON Z ((-84.4602665 42.2219816 0, -84.4591...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358,-84.4602665,42.2219816,"POLYGON ((-84.4601665 42.2219816, -84.46016698..."
4,. Kalamazoo Christian,"POLYGON Z ((-85.6134503 42.273406 0, -85.61346...","POLYGON Z ((-85.6134503 42.273406 0, -85.61452...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027,-85.6134503,42.273406,"POLYGON ((-85.6133503 42.273406, -85.613350781..."


In [71]:
### Import folium and create map
x = 40

home_pt = Point(df['home_pt'][x])

foul_poly = df['foul'][x]
fop_poly = df['fop'][x]

foul_geom = Polygon(foul_poly)
fop_geom = Polygon(fop_poly)
polygon = gpd.GeoDataFrame(index=[x], crs='epsg:3857', geometry=[fop_geom])       


print(polygon.boundary.distance(home_pt))

test_pt = polygon.exterior.interpolate(polygon.exterior.project(home_pt))

# print(test_pt)
# print(home_pt)
# print(test_pt.distance(home_pt))

# 1 METER = 3.28084 FEET


# import folium
# m = folium.Map(home, zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m

TypeError: 'Polygon' object is not iterable

In [26]:
x = 12

home = [df['home_lat'][x], df['home_long'][x]]

foul_poly = df['foul'][x]
fop_poly = df['fop'][x]

foul_geom = Polygon(foul_poly)
fop_geom = Polygon(fop_poly)

df['foul'].boundary.distance(home)

AttributeError: 'Series' object has no attribute 'boundary'

In [ ]:
# https://gis.stackexchange.com/questions/294206/how-to-create-a-simple-polygon-from-coordinates-in-geopandas-with-python

# from shapely import wkt

# import geopandas as gpd
# from shapely.geometry import Polygon

# lon_lat_list = df['foul'][3]
# polygon_geom = Polygon(lon_lat_list)
# polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])       

# import folium
# m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m
# df['foul_shapely'] = df['foul_str'].apply(lambda x: wkt.loads(x)) # only works with strings
# df.head()